In [1]:
#coding:utf-8
from SPARQLWrapper import SPARQLWrapper, JSON, XML, N3, RDF, CSV, TSV
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, SKOS, OWL, RDF, RDFS, VOID, XMLNS, XSD, SDO
from pandas import json_normalize
import pandas as pd

In [2]:
endPoint = "http://DESKTOP-5KN2LT0:7200/repositories/OERRepositorio"

In [3]:
sparql = SPARQLWrapper(endPoint)

In [32]:
queryIndexText="""
    PREFIX :<http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst:<http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX pubo: <http://ontology.ontotext.com/publishing#>
    PREFIX schema: <http://schema.org/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX vocab: <http://data.utpl.edu.ec/oer/vocab#>

    SELECT ?score ?tesis ?titulo ?resumen ?date ?nivelOer 
    (GROUP_CONCAT(DISTINCT ?autor; SEPARATOR=", ") AS ?autores)
    (GROUP_CONCAT(DISTINCT ?tutor; SEPARATOR=", ") AS ?tutores)
    (GROUP_CONCAT(DISTINCT ?parabrasClaver; SEPARATOR=", ") AS ?parabrasClave)
    ?formato ?idioma ?url {
        BIND ("%s" as ?parameters)
        ?search a inst:IndexText ;
            :searchTerm "%s";
            :searchParameters ?parameters;
            :documentResult ?result .
        ?result :value ?documentID ;
                :score ?score.
        ?tesis schema:mentions|schema:keywords|schema:mentions/dct:subject|schema:mentions/dct:subject/skos:broarder ?documentID .
        ?tesis schema:educationalLevel ?nivelO .
        ?nivelO schema:name ?name .
        filter(?parameters = ?name)
        minus{?tesis schema:abstract "NULL".}
        optional{?tesis schema:name ?titulo . }
        optional{?tesis schema:abstract ?resumen . }
        optional{?tesis schema:datePublisher ?date . }
        optional{?tesis schema:educationalLevel ?nivelOer . }
        optional{?tesis schema:author ?autor . }
        optional{?tesis schema:contributor ?tutor . }
        optional{?tesis schema:keywords ?parabrasClaver . }
        optional{?tesis schema:encondingFormat ?formato . }
        optional{?tesis schema:inLanguage ?idioma . }
        optional{?tesis schema:url ?url . }
    } group by ?score ?tesis ?titulo ?resumen ?date ?nivelOer ?formato ?idioma ?url
    order by ?nivelOer desc(?score)
    """
queryIndexText1="""
    PREFIX :<http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst:<http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX pubo: <http://ontology.ontotext.com/publishing#>
    PREFIX schema: <http://schema.org/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX vocab: <http://data.utpl.edu.ec/oer/vocab#>

    SELECT ?score ?tesis ?titulo ?resumen ?date ?nivelOer 
    (GROUP_CONCAT(DISTINCT ?autor; SEPARATOR=", ") AS ?autores)
    (GROUP_CONCAT(DISTINCT ?tutor; SEPARATOR=", ") AS ?tutores)
    (GROUP_CONCAT(DISTINCT ?parabrasClaver; SEPARATOR=", ") AS ?parabrasClaver)
    ?formato ?idioma ?url {
        BIND ("%s" as ?parameters)
        ?search a inst:IndexText ;
            :searchTerm "%s";
            :searchParameters ?parameters;
            :documentResult ?result .
        ?result :value ?documentID ;
                :score ?score.
        ?tesis schema:mentions|schema:keywords|schema:mentions/dct:subject|schema:mentions/dct:subject/skos:broarder ?documentID .
        ?tesis schema:educationalLevel ?nivelO .
        ?nivelO schema:name ?name .
        ?nivelU schema:name ?parameters .
        ?nivelU vocab:nextCompetencyLevel ?nextLU .
        filter(?parameters = ?name || ?nextLU = ?nextLO)
        optional{?tesis schema:educationalLevel+ ?nextLO .}
        optional{?tesis schema:name ?titulo . }
        optional{?tesis schema:abstract ?resumen . }
        optional{?tesis schema:datePublisher ?date . }
        optional{?tesis schema:educationalLevel ?nivelOer . }
        optional{?tesis schema:author ?autor . }
        optional{?tesis schema:contributor ?tutor . }
        optional{?tesis schema:keywords ?parabrasClaver . }
        optional{?tesis schema:encondingFormat ?formato . }
        optional{?tesis schema:inLanguage ?idioma . }
        optional{?tesis schema:url ?url . }
    } group by ?score ?tesis ?titulo ?resumen ?date ?nivelOer ?formato ?idioma ?url
    order by ?nivelOer desc(?score)
    """
    

In [33]:
def ejecutarConsulta(consulta, n, k):
    sparql.setQuery(consulta%(n,k))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    return results

In [34]:
def getIndexText(nivel, keyword):
    if(nivel!="Avanzado"):
        r = ejecutarConsulta(queryIndexText1, nivel, keyword)
    else:
        r = ejecutarConsulta(queryIndexText, nivel, keyword)
    
    return json_normalize(r["results"]["bindings"])

In [35]:
te = pd.read_excel("temasEvaluacion.xlsx")
count = len(te)
resultIndexText = pd.DataFrame()
for index, row in te.iterrows():
    if(count!=len(te)):
        r2 = getIndexText(row['Nivel'], row['Terminos'])
        r2['TermBusqueda'] = row['Terminos']
        resultIndexText = pd.concat([resultIndexText, r2])
    else:
        r1 = getIndexText(row['Nivel'], row['Terminos'])
        r1['TermBusqueda'] = row['Terminos']
        resultIndexText = r1
    count=-1 

In [36]:
m = resultIndexText[['score.value', 'tesis.value', 'titulo.value', 
             'resumen.value', 'date.value', 'nivelOer.value','parabrasClaver.value', 'TermBusqueda']]
m.head()
m.to_csv('resultadosIndexText.csv', index = False)

In [4]:
queryIndexPrediction1="""
    PREFIX :<http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst:<http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX psi:<http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX schema: <http://schema.org/>
    PREFIX vocab: <http://data.utpl.edu.ec/oer/vocab#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT distinct ?entity ?score ?Titulo
    ?resumen ?date ?nivelOer 
    (GROUP_CONCAT(DISTINCT ?autor; SEPARATOR=", ") AS ?autores)
    (GROUP_CONCAT(DISTINCT ?tutor; SEPARATOR=", ") AS ?tutores)
    (GROUP_CONCAT(DISTINCT ?parabrasClaver; SEPARATOR=", ") AS ?parabrasClave)
    ?formato ?idioma ?url
    {
        BIND ("%s" as ?parameters)
        ?search a inst:IndexPredication ;
            psi:searchEntity <%s>;
            psi:searchPredicate <http://www.ontotext.com/graphdb/similarity/psi/any>;
            :searchParameters ?parameters;
            psi:entityResult ?result .
        ?result :value ?entity ;
                :score ?score .
        ?entity rdf:type schema:Thesis; schema:name ?Titulo.
        ?entity schema:educationalLevel ?nivelO .
        ?nivelO schema:name ?name .
        ?nivelU schema:name ?parameters .
        ?nivelU vocab:nextCompetencyLevel ?nextLU .
        optional{?entity schema:educationalLevel+ ?nextLO .}
        filter(?parameters = ?name || ?nextLU = ?nextLO)
        optional{?entity schema:abstract ?resumen . }
        optional{?entity schema:datePublisher ?date . }
        optional{?entity schema:educationalLevel ?nivelOer . }
        optional{?entity schema:author ?autor . }
        optional{?entity schema:contributor ?tutor . }
        optional{?entity schema:keywords ?parabrasClaver . }
        optional{?entity schema:encondingFormat ?formato . }
        optional{?entity schema:inLanguage ?idioma . }
        optional{?entity schema:url ?url . }
    }group by ?entity ?score ?tesis ?Titulo ?resumen ?date ?nivelOer ?formato ?idioma ?url
    order by ?nivelOer desc(?score)
    """
queryIndexPrediction = """
    PREFIX :<http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst:<http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX psi:<http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX schema: <http://schema.org/>
    PREFIX vocab: <http://data.utpl.edu.ec/oer/vocab#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT distinct ?entity ?score ?Titulo
    ?resumen ?date ?nivelOer 
    (GROUP_CONCAT(DISTINCT ?autor; SEPARATOR=", ") AS ?autores)
    (GROUP_CONCAT(DISTINCT ?tutor; SEPARATOR=", ") AS ?tutores)
    (GROUP_CONCAT(DISTINCT ?parabrasClaver; SEPARATOR=", ") AS ?parabrasClave)
    ?formato ?idioma ?url
    {
        BIND ("%s" as ?parameters)
        ?search a inst:IndexPredication ;
            psi:searchEntity <%s>;
            psi:searchPredicate <http://www.ontotext.com/graphdb/similarity/psi/any>;
            :searchParameters ?parameters;
            psi:entityResult ?result .
        ?result :value ?entity ;
                :score ?score .
        ?entity rdf:type schema:Thesis; schema:name ?Titulo.
        ?entity schema:educationalLevel ?nivelO .
        ?nivelO schema:name ?name .
        filter(?parameters = ?name)
        minus{?tesis schema:abstract "NULL".}
        optional{?entity schema:abstract ?resumen . }
        optional{?entity schema:datePublisher ?date . }
        optional{?entity schema:educationalLevel ?nivelOer . }
        optional{?entity schema:author ?autor . }
        optional{?entity schema:contributor ?tutor . }
        optional{?entity schema:keywords ?parabrasClaver . }
        optional{?entity schema:encondingFormat ?formato . }
        optional{?entity schema:inLanguage ?idioma . }
        optional{?entity schema:url ?url . }
    }group by ?entity ?score ?tesis ?Titulo ?resumen ?date ?nivelOer ?formato ?idioma ?url
    order by ?nivelOer desc(?score)
"""

In [5]:
def ejecutarConsultaIP(consultaIP, n, k):
    sparql.setQuery(consultaIP%(n,k))
    sparql.setReturnFormat(JSON)
    resultsIP = sparql.query().convert()
    
    return resultsIP

In [6]:
def getIndexPrediction(nivel, URI):
    if(nivel!="Avanzado"):
        rIP = ejecutarConsultaIP(queryIndexPrediction1, nivel, URI)
    else:
        rIP = ejecutarConsultaIP(queryIndexPrediction, nivel, URI)
    
    return json_normalize(rIP["results"]["bindings"])

In [11]:
URIS = pd.read_excel('urisEvaluacion.xlsx') 
cont = len(URIS)
resultIP1 = pd.DataFrame()
for index, row in URIS.iterrows():
    if(cont!=len(URIS)):
        r2 = getIndexPrediction(row['nivel'], row['tesis'])
        r2['uriTem'] = row['topic']
        r2['uriBus'] = row['tesis']
        resultIP1 = pd.concat([resultIP1, r2])
    else:
        r1 = getIndexPrediction(row['nivel'], row['tesis'])
        r1['uriTem'] = row['topic']
        r1['uriBus'] = row['tesis']
        resultIP1 = r1
    cont=-1 

In [13]:
rIP = resultIP1[['score.value', 'Titulo.value', 'resumen.value', 'date.value','nivelOer.value', 'parabrasClave.value','uriBus' ,'uriTem']]
rIP.head()

,score.value,Titulo.value,resumen.value,date.value,nivelOer.value,parabrasClave.value,uriBus,uriTem
0,0.4103570692188991,Diseñar y gestionar servicios de valor agregad...,NULL,2012,http://data.utpl.edu.ec/oer/Avanzado,http://data.utpl.edu.ec/oer/Ingeniero_en_Siste...,http://data.utpl.edu.ec/oer/Aplicacion_de_sema...,Resource_Description_Framework
1,0.3552707317302465,Sistema de gestion de bibliotecas Zebra,NULL,2012,http://data.utpl.edu.ec/oer/Avanzado,http://data.utpl.edu.ec/oer/Ingeniero_en_Siste...,http://data.utpl.edu.ec/oer/Aplicacion_de_sema...,Resource_Description_Framework
2,0.3064814536314808,Aplicacion de tecnicas de Linked Data para la ...,El presente proyecto de investigación se desar...,2014,http://data.utpl.edu.ec/oer/Avanzado,http://data.utpl.edu.ec/oer/Ingeniero_en_Siste...,http://data.utpl.edu.ec/oer/Aplicacion_de_sema...,Resource_Description_Framework
3,0.30568074292726155,Desarrollo de software de redgrupo de telecomu...,NULL,2012,http://data.utpl.edu.ec/oer/Avanzado,http://data.utpl.edu.ec/oer/Ingeniero_en_Siste...,http://data.utpl.edu.ec/oer/Aplicacion_de_sema...,Resource_Description_Framework
4,0.2895248840335161,Sistema para la distribucion evaluacion de inf...,NULL,2012,http://data.utpl.edu.ec/oer/Avanzado,"http://data.utpl.edu.ec/oer/Web, http://data.u...",http://data.utpl.edu.ec/oer/Aplicacion_de_sema...,Resource_Description_Framework


In [14]:
rIP.to_csv('resultsIndexPre.csv', index = False)
pd.read_csv("resultsIndexPre.csv", parse_dates=[0])

,score.value,Titulo.value,resumen.value,date.value,nivelOer.value,parabrasClave.value,uriBus,uriTem
0,0.4103570692188991,Diseñar y gestionar servicios de valor agregad...,NaN,2012,http://data.utpl.edu.ec/oer/Avanzado,http://data.utpl.edu.ec/oer/Ingeniero_en_Siste...,http://data.utpl.edu.ec/oer/Aplicacion_de_sema...,Resource_Description_Framework
1,0.3552707317302465,Sistema de gestion de bibliotecas Zebra,NaN,2012,http://data.utpl.edu.ec/oer/Avanzado,http://data.utpl.edu.ec/oer/Ingeniero_en_Siste...,http://data.utpl.edu.ec/oer/Aplicacion_de_sema...,Resource_Description_Framework
2,0.3064814536314808,Aplicacion de tecnicas de Linked Data para la ...,El presente proyecto de investigación se desar...,2014,http://data.utpl.edu.ec/oer/Avanzado,http://data.utpl.edu.ec/oer/Ingeniero_en_Siste...,http://data.utpl.edu.ec/oer/Aplicacion_de_sema...,Resource_Description_Framework
3,0.30568074292726155,Desarrollo de software de redgrupo de telecomu...,NaN,2012,http://data.utpl.edu.ec/oer/Avanzado,http://data.utpl.edu.ec/oer/Ingeniero_en_Siste...,http://data.utpl.edu.ec/oer/Aplicacion_de_sema...,Resource_Description_Framework
4,0.2895248840335161,Sistema para la distribucion evaluacion de inf...,NaN,2012,http://data.utpl.edu.ec/oer/Avanzado,"http://data.utpl.edu.ec/oer/Web, http://data.u...",http://data.utpl.edu.ec/oer/Aplicacion_de_sema...,Resource_Description_Framework
...,...,...,...,...,...,...,...,...
244,0.29777576188975274,Implementacion de una herramienta generadora d...,A traves del presente trabajo se detalla el pr...,2013,http://data.utpl.edu.ec/oer/Intermedio,"http://data.utpl.edu.ec/oer/Medico, http://dat...",http://data.utpl.edu.ec/oer/Determinar_el_nive...,Base_de_datos
245,0.2878724765104046,Incidencia del uso de tecnologias en las activ...,Se busca recoger información que permita deter...,2012,http://data.utpl.edu.ec/oer/Intermedio,http://data.utpl.edu.ec/oer/Ingeniero_en_infor...,http://data.utpl.edu.ec/oer/Determinar_el_nive...,Base_de_datos
246,0.2702359578420974,Comunicacion entre plataformas Windows y Linux...,"En los hogares y oficinas de todo el mundo, la...",2012,http://data.utpl.edu.ec/oer/Intermedio,http://data.utpl.edu.ec/oer/Ingeniero_en_siste...,http://data.utpl.edu.ec/oer/Determinar_el_nive...,Base_de_datos
247,0.2526914717926431,Desarrollo del laboratorio virtual de hidrologia,El presente trabajo realizado constituyo en la...,2013,http://data.utpl.edu.ec/oer/Intermedio,http://data.utpl.edu.ec/oer/Ingeniero_en_Siste...,http://data.utpl.edu.ec/oer/Determinar_el_nive...,Base_de_datos


In [39]:
URIS = pd.read_excel('urisEvaluacion.xlsx') 

,tesis,topic,nivel
0,http://data.utpl.edu.ec/oer/Aplicacion_de_sema...,Resource_Description_Framework,Intermedio
1,http://data.utpl.edu.ec/oer/Publicacion_de_dat...,Resource_Description_Framework,Intermedio
2,http://data.utpl.edu.ec/oer/Analisis_de_redes_...,Resource_Description_Framework,Principiante
3,http://data.utpl.edu.ec/oer/Extraccion_de_info...,Resource_Description_Framework,Principiante
4,http://data.utpl.edu.ec/oer/Medicion_de_Accesi...,Recursos_educativos_abiertos,Intermedio
5,http://data.utpl.edu.ec/oer/Aplicacion_de_tecn...,Recursos_educativos_abiertos,Avanzado
6,http://data.utpl.edu.ec/oer/Creacion_de_recurs...,Recursos_educativos_abiertos,Intermedio
7,http://data.utpl.edu.ec/oer/Propuesta_e_implem...,Recursos_educativos_abiertos,Avanzado
8,http://data.utpl.edu.ec/oer/Analisis_de_las_in...,Minerí­a_de_datos,Intermedio
9,http://data.utpl.edu.ec/oer/Tecnicas_de_mineri...,Minerí­a_de_datos,Principiante
